### 2. Create BERT Embedding from all layers

In [1]:
# 라이브러리 설치
# pip install transformers

In [1]:
# 라이브러리
import torch
from transformers import BertModel, BertTokenizer

In [2]:
# BERT 모델 불러오기
model = BertModel.from_pretrained('bert-base-uncased', 
                                  output_hidden_states = True)

- bert-based-uncased를 사용한다. 
- 모든 토큰을 소문자화한 모델이다.
- 임베딩 벡터의 크기는 768이다.
- ```output_hidden_states=True```로 설정해서 모든 인코더 레이어에서 임베딩을 얻는다.

In [3]:
# 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

이제 문장을 토큰화하고 ID를 얻는 전처리 과정을 진행하자.

In [4]:
# 입력 문장
sentence = 'I love Paris'

# 문장 토큰화
tokens = tokenizer.tokenize(sentence)

# 확인
print(tokens)

['i', 'love', 'paris']


- 사전 학습된 토크나이저가 문장을 토큰 단위로 분리한다.
- 이번에는 특수 토큰을 추가한다.

In [5]:
# 특수 토큰 추가
tokens = ['[CLS]'] + tokens + ['[SEP]']

print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]']


In [6]:
# 패딩 추가
tokens = tokens + ['[PAD]'] + ['[PAD]']

print(tokens)

['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']


In [7]:
# 어텐션 마스크 생성
attn_mask = [0 if i == '[PAD]' else 1 for i in tokens]

print(attn_mask)

[1, 1, 1, 1, 1, 0, 0]


In [8]:
# 토큰 ID 생성
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(token_ids)

[101, 1045, 2293, 3000, 102, 0, 0]


- 사전 학습된 토크나이저가 토큰을 ID로 매핑했다.
- 이제 ID와 어텐션 마스크를 텐서로 만들어주자.

In [9]:
# 텐서화
token_ids = torch.tensor(token_ids).unsqueeze(0)
attn_mask = torch.tensor(attn_mask).unsqueeze(0)

print(token_ids)
print(attn_mask)

tensor([[ 101, 1045, 2293, 3000,  102,    0,    0]])
tensor([[1, 1, 1, 1, 1, 0, 0]])


이제 임베딩을 추출해보자.

In [10]:
# 임베딩 가져오기
last_hidden_state, pooler_output, hidden_states = model(token_ids, attention_mask = attn_mask, return_dict = False)

model은 튜플 형태로 3가지 값을 반환한다.
- ```last_hidden_state``` : 최종 인코더 계층(12번째 인코더)에서 얻은 표현
- ```pooler_output``` : 최종 인코더 계층(12번째 인코더)의 [CLS] 토큰 표현
- ```hidden_states``` : 모든 인코더 계층에서 얻은 표현

😅 transformers 3.xx 버전에서는 튜플 형태를 명시적으로 출력해달라고 해야 한다. 따라서 ```return_dict=False```로 설정한다.
- 참고 : [stackoverflow](https://stackoverflow.com/questions/66524542/attributeerror-str-object-has-no-attribute-shape-while-encoding-tensor-usin)

In [11]:
# 모든 토큰의 임베딩 표현 확인
print(last_hidden_state.shape)

torch.Size([1, 7, 768])


```last_hidden_state```는 각 토큰의 임베딩 벡터를 출력하게 된다.
- [배치 크기, 시퀀스 길이, 은닉 상태 크기]를 나타낸다.
- **배치 크기** : 여기서는 1이다. 만약 여러개 문장을 사용했다면, 배치 크기가 늘어나게 된다.
- **시퀀스 길이** : 문장의 길이, 토큰의 개수를 나타낸다.
- **은닉 상태 크기** : 임베딩 벡터의 크기로, bert-base 모델은 768이다.

😁 마지막 계층의 은닉 상태를 얻는 부분이라 이전과 동일하다!

또한 다음과 같이 각 토큰의 표현을 얻을 수 있다.
- ```last_hidden_state[0][0]```은 첫번째 배치의 첫번째 시퀀스이므로 '[CLS]'의 임베딩 벡터이다.
- ```last_hidden_state[0][1]```은 첫번째 배치의 두번째 시퀀스이므로 'I'의 임베딩 벡터이다.

In [12]:
# love의 임베딩 벡터 확인
print(last_hidden_state[0][2])

tensor([ 1.0410e+00,  7.7545e-01,  1.0335e+00, -2.5202e-01,  3.0039e-01,
         8.8043e-03, -2.2426e-01,  2.6048e-01, -5.2928e-01, -4.6895e-01,
        -4.2587e-01, -5.7644e-01, -4.5401e-01,  6.2828e-01, -1.0082e-01,
         3.2514e-01,  3.8628e-01,  2.0141e-01,  4.4131e-01,  4.3412e-01,
         3.7723e-01,  5.1090e-01, -3.7849e-01,  1.2689e-01,  1.2464e-01,
        -7.2938e-01, -3.5591e-01, -8.4662e-02,  5.2243e-01,  8.9538e-02,
         3.9896e-01,  6.2042e-01, -4.3549e-02, -1.3473e-01, -5.1585e-01,
         3.8026e-01, -4.8698e-01,  2.8725e-01, -5.1415e-01,  2.4158e-01,
         4.5858e-02, -1.1117e-01,  1.3997e-01, -3.4089e-01,  6.4368e-01,
        -7.3179e-01,  2.6873e-01,  6.0686e-01,  1.8135e+00, -5.0584e-01,
         1.0304e-01,  2.7961e-01, -5.5215e-02,  4.5086e-01, -6.9957e-02,
         1.1098e+00, -8.3400e-01, -8.3123e-01, -2.0023e-01,  1.0537e-01,
        -4.8093e-01, -7.0247e-01,  1.4388e-01, -3.7023e-01, -2.4109e-01,
         6.4811e-01, -3.5716e-01,  2.9457e-01, -6.2

In [13]:
# CLS 토큰의 표현 확인
print(pooler_output.shape)

torch.Size([1, 768])


```[CLS]``` 토큰의 값을 출력한다.
- [배치 크기, 은닉 상태 크기]를 나타낸다.
- ```[CLS]``` 토큰을 문장의 표현 벡터로 사용할 수 있다.

😁 역시나 동일하게 ```[CLS]```의 표현을 얻는 부분이라 이전과 동일하다!

In [14]:
# 모든 인코더 계층 확인
len(hidden_states)

13

입력 임베딩 레이어($h_0$)에서 최종 인코더 레이어($h_{12}$)까지를 포함해서 13개의 값을 포함하는 튜플이다.
- ```hidden_states[0]```은 입력 임베딩 레이어에서 얻은 모든 토큰의 표현 벡터
- ```hidden_states[3]```은 세 번째 인코더 계층에서 얻은 모든 토큰의 표현 벡터

In [15]:
print(hidden_states[0].shape)

torch.Size([1, 7, 768])


입력 임베딩 레이어에서 얻은 모든 토큰의 표현 벡터이다.
- [배치 크기, 시퀀스 길이, 은닉 상태 크기]를 나타낸다.

In [17]:
print(hidden_states[3].shape)

torch.Size([1, 7, 768])


3번째 인코더 레이어에서 얻은 모든 토큰의 표현 벡터이다.